In [2]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json

1. json 파일에서 bbox별로 score, 크기 가져오기

In [16]:
# JSON 파일 경로
json_file_path = 'bbox_result.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    bbox_data = json.load(f)

print(f'개수: {len(bbox_data)}')

개수: 104125


In [17]:
bbox_data[0].keys()

dict_keys(['image_file', 'center', 'scale', 'rotation', 'bbox', 'bbox_score', 'dataset', 'joints_3d', 'joints_3d_visible', 'bbox_id'])

In [18]:
bbox_df = pd.DataFrame(bbox_data)[['image_file', 'bbox', 'bbox_score','bbox_id']]
bbox_df

,image_file,bbox,bbox_score,bbox_id
0,data/coco/val2017/000000532481.jpg,"[249.8199079291458, 175.21093805640606, 74.004...",0.999274,0
1,data/coco/val2017/000000532481.jpg,"[284.07836701140275, 370.43747059580454, 13.84...",0.983319,1
2,data/coco/val2017/000000532481.jpg,"[244.05099093550018, 170.61324112294565, 74.56...",0.471152,2
3,data/coco/val2017/000000532481.jpg,"[285.7852415208732, 370.21400193144893, 10.915...",0.427866,3
4,data/coco/val2017/000000532481.jpg,"[254.66838783379924, 177.22587387050734, 65.52...",0.212189,4
...,...,...,...,...
104120,data/coco/val2017/000000179112.jpg,"[25.86279031631016, 11.718955167445618, 324.48...",0.000218,104120
104121,data/coco/val2017/000000179112.jpg,"[155.69763757619663, 195.84793595792362, 187.8...",0.000108,104121
104122,data/coco/val2017/000000179112.jpg,"[25.7279295752435, 0.8558799166679145, 285.103...",0.000066,104122
104123,data/coco/val2017/000000311295.jpg,"[227.37860456174386, 169.90725877096875, 5.060...",0.033784,104123


In [19]:
bbox_df['image_file'] = bbox_df['image_file'].str.replace('val2017/', '').str.replace('.jpg', '').astype(int)

In [20]:
bbox_df

,image_file,bbox,bbox_score,bbox_id
0,532481,"[249.8199079291458, 175.21093805640606, 74.004...",0.999274,0
1,532481,"[284.07836701140275, 370.43747059580454, 13.84...",0.983319,1
2,532481,"[244.05099093550018, 170.61324112294565, 74.56...",0.471152,2
3,532481,"[285.7852415208732, 370.21400193144893, 10.915...",0.427866,3
4,532481,"[254.66838783379924, 177.22587387050734, 65.52...",0.212189,4
...,...,...,...,...
104120,179112,"[25.86279031631016, 11.718955167445618, 324.48...",0.000218,104120
104121,179112,"[155.69763757619663, 195.84793595792362, 187.8...",0.000108,104121
104122,179112,"[25.7279295752435, 0.8558799166679145, 285.103...",0.000066,104122
104123,311295,"[227.37860456174386, 169.90725877096875, 5.060...",0.033784,104123


In [21]:
# bbox_df.to_csv("bbox_df.csv",index=False)

In [ ]:
bbox_df = pd.DataFrame(bbox_data)[['image_file', 'bbox', 'bbox_score','bbox_id']]
bbox_df

In [3]:
# JSON 파일 경로
json_file_path = 'kpts_result.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    kpts_data = json.load(f)

print(f'개수: {len(kpts_data)}')

개수: 3893


In [8]:
df_list = []
for image_id, image_data in kpts_data.items():
    for data in image_data:
        df_list.append(pd.DataFrame({
            "image_id": [data["image_id"]],
            "bbox_id": [data["bbox_id"]],
            "area": [data["area"]],
            "score": [data["score"]],
            "keypoints": [data["keypoints"]],
            "center": [data["center"]],
            "scale": [data["scale"]],
        }))

# DataFrame을 합치기
kpts_df = pd.concat(df_list, ignore_index=True)

In [9]:
kpts_df.columns

Index(['image_id', 'bbox_id', 'area', 'score', 'keypoints', 'center', 'scale'], dtype='object')

In [10]:
kpts_df = kpts_df[['image_id', 'area','score']]

In [12]:
# kpts_df.to_csv("kpt_df.csv", index=False)

In [72]:
bbox_df["width"] = bbox_df["bbox"].apply(lambda bbox: bbox[2])
bbox_df["height"] = bbox_df["bbox"].apply(lambda bbox: bbox[3])
bbox_df["size"] = bbox_df["width"]*bbox_df["height"]